In [5]:
import math
from typing import List, Tuple

import numpy as np

ALL_BUTTONS: str = "123456789*0#"

In [6]:
def compute_laziest_path(telephone_number: str) -> Tuple[float, List[Tuple[str]]]:
    # Computing auxilary arrays with all partial minimal distances
    left_array, right_array = compute_distance_arrays(telephone_number)
    laziest_distance = min(min(left_array[-1, :]), min(right_array[-1, :]))
    laziest_path = [("*", "#")]
    for ix, key in enumerate(telephone_number):
        left_array_ix = left_array[ix, :]
        right_array_ix = right_array[ix, :]
        
        if min(left_array_ix) <= min(right_array_ix):
            ix_min = np.argmin(left_array_ix)
            button_min = ALL_BUTTONS[ix_min]
            laziest_path_step_ix = (key, button_min)
        else:
            ix_min = np.argmin(right_array_ix)
            button_min = ALL_BUTTONS[ix_min]
            laziest_path_step_ix = (button_min, key)
        
        laziest_path.append(laziest_path_step_ix)
        
    return laziest_distance, laziest_path

def compute_distance_arrays(telephone_number: str) -> Tuple[np.ndarray, np.ndarray]:
    N = len(telephone_number)
    K = len(ALL_BUTTONS)

    left_array = np.zeros((N, K))
    right_array = np.zeros((N, K))

    # Filling in first row of left array:
    # First addend: distance while moving left finger from initial "*" to first key of telephone number
    # Second addend: vector of distances while moving right finger from initial "#" to all possible buttons
    left_array[0, :] = [
        euclidean_distance_between_buttons("*", telephone_number[0])
        + euclidean_distance_between_buttons("#", button) for button in ALL_BUTTONS
    ]
    # First row of right array is filled in analogously
    right_array[0, :] = [
        euclidean_distance_between_buttons("#", telephone_number[0])
        + euclidean_distance_between_buttons("*", button) for button in ALL_BUTTONS
    ]

    # Filling in (1, ..., N) rows of left and right array recursively"
    
    # Looping through keys of telephone number
    for ix_n in range(N - 1):
        # Looping through all possible buttons
        for ix_k, button_k in enumerate(ALL_BUTTONS):
            left_array_ix_n = left_array[ix_n, :]
            right_array_ix_n = right_array[ix_n, :]
            
            # Array with distances from k-th button to all possible buttons
            dist_from_k_array = np.array(
                [euclidean_distance_between_buttons(button, button_k) for button in ALL_BUTTONS]
            )
            # Array with distances from (n + 1)-th key of telephone number to all possible buttons
            dist_from_ix_n_array = np.array(
                [euclidean_distance_between_buttons(button, telephone_number[ix_n + 1]) for button in ALL_BUTTONS]
            )
            
            # Filling in (n + 1)-th row and k-th column of left array
            # Minimum of distances of all paths where n-th key was dialed using left finger
            min_left_left = min(
                left_array_ix_n
                + dist_from_k_array
                + euclidean_distance_between_buttons(telephone_number[ix_n], telephone_number[ix_n + 1])
            )
            # Minimum of distances of all paths where n-th key was dialed using right finger
            min_left_right = min(
                right_array_ix_n
                + dist_from_ix_n_array
                + euclidean_distance_between_buttons(telephone_number[ix_n], button_k)
            )

            min_left = min(min_left_left, min_left_right)

            # Filling in (n + 1)-th row and k-th column of right array
            min_right_right = min(
                right_array_ix_n
                + dist_from_k_array
                + euclidean_distance_between_buttons(telephone_number[ix_n], telephone_number[ix_n + 1])
            )
            min_right_left = min(
                left_array_ix_n
                + dist_from_ix_n_array
                + euclidean_distance_between_buttons(telephone_number[ix_n], button_k)
            )

            min_right = min(min_right_right, min_right_left)

            left_array[ix_n + 1, ix_k] = min_left
            right_array[ix_n + 1, ix_k] = min_right
            
    return left_array, right_array

def euclidean_distance_between_buttons(button_0: str, button_1: str) -> float:
    position_0 = translate_button_to_position(button_0)
    position_1 = translate_button_to_position(button_1)
    return euclidean_distance_between_positions(position_0, position_1)

def translate_button_to_position(button: str) -> Tuple[int, int]:
    button_ix = ALL_BUTTONS.index(button)
    position = (button_ix // 3, button_ix % 3)
    return position
    
def euclidean_distance_between_positions(position_0: Tuple[int, int], position_1: Tuple[int, int]) -> float:
    return math.sqrt((position_0[0] - position_1[0]) ** 2 + (position_0[1] - position_1[1]) ** 2)


In [7]:
compute_laziest_path("110")

(4.0, [('*', '#'), ('1', '#'), ('1', '#'), ('1', '0')])

In [8]:
compute_laziest_path("94201")

(6.650281539872885,
 [('*', '#'), ('*', '9'), ('4', '9'), ('2', '9'), ('0', '2'), ('0', '1')])

Time complexity of given solution is O(nk^2), space complexity is O(nk).